In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
import uncertainties as unc
import scipy.special as ss
import xarray as xr

from tqdm.notebook import tqdm

import sys
sys.path.append('../')
import general_purpose.utilities as ut
import general_purpose.uplotlib as uplt
import general_purpose.cartopy_plots as cplt

from importlib import reload

import rea.reconstruct as rec

# View the files

In [ ]:
folder = '../veros-noise-product/atlantic2'

In [ ]:
# eofs_re = np.load('EOFs_new.npy')
eofs_re = np.load(f'{folder}/EOFs.npy')
eofs_re.shape

In [ ]:
pcs_re = np.load(f'{folder}/PCs.npy')
pcs_re.shape

In [ ]:
lm = np.load(f'{folder}/landMask.npy')
lm.shape

In [ ]:
lags = np.load(f'{folder}/Lags.npy')
lags.shape

In [ ]:
yw_rho = np.load(f'{folder}/yw_rho.npy')
yw_sigma = np.load(f'{folder}/yw_sigma.npy')

yw_rho.shape, yw_sigma.shape

In [ ]:
plt.figure()
plt.contourf(eofs_re[0])
plt.colorbar()

In [ ]:
plt.figure()
plt.plot(yw_sigma)

In [ ]:
plt.figure()
plt.plot(lags)

In [ ]:
yw_rho.shape

In [ ]:
eof_id = 6

plt.close(4)
plt.figure(num=4)
_ = plt.plot(np.arange(yw_rho.shape[1]), yw_rho[eof_id - 1], marker='o')

### Plot the EOFs with the proper coordinates

In [ ]:
av = xr.load_dataset('../veros-temp-noise-old/__test__/e1.0000.averages.nc')

In [ ]:
av

In [ ]:
av['yt']

In [ ]:
LON, LAT = np.meshgrid(av['xt'] - 360, av['yt'])

In [ ]:
plt.close(2)
fig = plt.figure(num=2, figsize=(8,4))
ax = fig.add_subplot(111, projection=cplt.ccrs.PlateCarree())

cplt.geo_plotter(ax, LON, LAT,
                 # eofs_re[0][::-1,:],
                 lm[::-1],
                 greenwich=True,
                 draw_gridlines=False, draw_labels=False,
                 put_colorbar=False
                )

In [ ]:
reload(cplt)

In [ ]:
vmax = np.nanmax(np.abs(eofs_re))
ofl = (8,11)
plots_per_fig = np.prod(ofl)
start = 48
plt.close('all')
i = 0
while i < 1:
    lower = start + plots_per_fig*i
    upper = min(start + plots_per_fig*(i+1), eofs_re.shape[0])
    if upper <= lower:
        break
    ims = cplt.mfp(LON, LAT, eofs_re[lower:upper].transpose(1,2,0)[::-1,:,:],
                   projections=cplt.ccrs.PlateCarree(), extents=None, titles=list(range(lower + 1,upper + 1)),
                   figsize=(18,9),
                   # figsize=(12,6),
                   fig_num = 8 + i,
                   one_fig_layout=ofl,
                   # mx=vmax,
                   # colorbar='shared',
                   colorbar='individual',
                   put_colorbar=False,
                   # apply_tight_layout=False
                  )
    i += 1

# Effect of noise on the dynamics

In [ ]:
def get_run(folder, iteration=None):
    if iteration is None:
        run = {'folder': folder, 'rec_dict': ut.json2dict(f'{folder}/reconstructed.json')}
    else:
        run = {'folder': folder, 'rec_dict': rec.reconstruct(f'{folder}/i{iteration:04d}')}
    
    run['abs_observables'] = {}
    run['rel_observables'] = {}
    for rname, r in run['rec_dict']['members'].items():
        last_segment_traj = np.load(f"{folder}/{run['rec_dict']['folders'][-1]}/{r['ancestry'][-1]}-traj.npy")
        first_segment_traj = np.load(f"{folder}/{run['rec_dict']['folders'][0]}/{r['ancestry'][0]}-traj.npy")
        run['abs_observables'][rname] = last_segment_traj[-1,1]
        run['rel_observables'][rname] = last_segment_traj[-1,1] - first_segment_traj[0,1]
        
    first_segment_traj = np.load(f"{folder}/{run['rec_dict']['folders'][0]}/{r['ancestry'][0]}-traj.npy")
    run['duration'] = last_segment_traj[-1,0] - first_segment_traj[0,0]
        
    return run

def compute_trajectories(run):
    folder = run['folder']
    rec_dict = run['rec_dict']
    rec_trajs = {}

    for rname, r in tqdm(rec_dict['members'].items()):
        traj = []
        for subfolder,e in zip(rec_dict['folders'], r['ancestry']):
            traj.append(np.load(f'{folder}/{subfolder}/{e}-traj.npy'))
        traj = np.concatenate(traj)

        rec_trajs[rname] = traj
    
    run['traj'] = rec_trajs
    
    return rec_trajs

def plot_traj(run, start=0, **kwargs):
    if 'traj' not in run:
        compute_trajectories(run)
    for traj in run['traj'].values():
        plt.plot(traj[start:,0]-traj[0,0], -traj[start:,1], **kwargs)

def expectation(run, func, mode='abs', **kwargs):
    _f = np.array([func(x, **kwargs) for x in run[f'{mode}_observables'].values()])
    _w = np.array([r['weight'] for r in run['rec_dict']['members'].values()])
    # _w = np.exp(-np.array([r['cum_log_escore'] for r in rec_dict['members'].values()]))
    # _n = np.exp(rec_dict['cum_log_norm_factor'])

    mean = np.mean(_f*_w)
    std = np.sqrt(np.mean((_f*_w)**2)/len(_w))

    return unc.ufloat(mean,std)

def f(x, a):
    return float(x>a)

def get_infos(folder):
    i = 0
    infos = {}
    while True:
        try:
            infos[i] = ut.json2dict(f'{folder}/i{i:04d}/info.json')
            i += 1
        except FileNotFoundError:
            break
            
    return infos


In [ ]:
## atlantic noise
# runs = {
#     '0': get_run('../veros/__test__/r-0_minimal'),
#     '1': get_run('__test__/rAtlN_s1-k0_minimal/'),
#     '2': get_run('__test__/rAtlN_s2-k0_minimal/'),
#     '5': get_run('__test__/rAtlN_s5-k0_minimal/')
# }

# atlantic noise, yearly resolution
control = np.load('../veros/__test__/amoc_max_f8_2_yearly.npy')
control[0] -= control[0,0]
control[1] *= -1
i = 0
runs = {
    # '0': {'traj': {'r1': control.T[200:400]}},
    '0': {'traj': {'r1': control.T[i:i+500]}},
    # '1': get_run('__test__/rY1AAtlN_s1-k0_minimal/'),
    '1': get_run('__test__/rY1AtlN_s1-k0_minimal/'),
}

## ocean noise (wrong)
# runs = {
#     '0': get_run('../veros/__test__/r-0_minimal'),
#     '1': get_run('__test__/rN_s1-k0_minimal/'),
#     '2': get_run('__test__/rN_s2-k0_minimal'),
#     '5': get_run('__test__/rN_s5-k0_minimal')
# }

## global noise (wrong)
# runs = {
#     '0': get_run('../veros/__test__/r-0_minimal'),
#     '1': get_run('../veros-temp-noise-old/__test__/rn-0_minimal/'),
#     '2': get_run('../veros-temp-noise-old/__test__/rn_s2-0_minimal'),
#     '5': get_run('../veros-temp-noise-old/__test__/rn_s5-0_minimal')
# }

# for run in runs.values():
#     compute_trajectories(run)

## Plot the trajectories

In [ ]:
runs['0']['traj']['r1']

In [ ]:
plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

for s,run in runs.items():
    plot_traj(run, label=fr'$\sigma = {s}$')

plt.legend()

plt.xlabel('time [yr]')
plt.ylabel('AMOC strength [Sv]')

# plt.xlim(0,500)

fig.tight_layout()

In [ ]:
# fig.savefig('../../papers/PhD_Thesis/manuscript/Figures/REA-VerOS/control-v-sigma.pdf')
fig.savefig('../../papers/PhD_Thesis/manuscript/Figures/REA-VerOS/control-v-sigma-Y1.pdf')

In [ ]:
stds = {s: np.std(run['traj']['r1'][:,1]) for s,run in runs.items()}
stds

In [ ]:
plt.close(7)
fig,ax = plt.subplots(num=7, figsize=(9,6))

plt.plot([int(s) for s in stds], stds.values(), marker='o')

plt.xlabel(r'Noise rescaling factor ($\sigma$)')
plt.ylabel('AMOC std [Sv]')

fig.tight_layout()

In [ ]:
import statsmodels.tsa.stattools as sss

In [ ]:
lags = 100
yres = 1
acfs = {s: sss.acf(run['traj']['r1'][:,1], nlags=lags//yres-1) for s,run in runs.items()}

In [ ]:
plt.close(8)
fig,ax = plt.subplots(num=8, figsize=(9,6))

for s,acf in acfs.items():
    plt.plot(np.arange(lags)[::yres], acf, label=fr'$\sigma = {s}$')

plt.axhline(0, linestyle='dashed', color='black')

plt.xlabel('Lag [yr]')
plt.ylabel('Autocorrelation')
plt.legend()

fig.tight_layout()

In [ ]:
plt.figure()
plt.plot(np.arange(lags)[::yres], sss.acf(runs['0']['traj']['r1'][:100,1], nlags=lags//yres-1))
plt.plot(np.arange(lags)[::yres], sss.acf(runs['1']['traj']['r1'][:,1], nlags=lags//yres-1))

### Power spectra

In [ ]:
data = runs['1']['traj']['r1'][:,1]
data = data - np.mean(data)
ps = np.abs(np.fft.fft(data))**2

time_step = 1
freqs = np.fft.fftfreq(data.size, time_step)

ps = ps[1:len(ps)//2]
freqs = freqs[1:len(freqs)//2]

plt.close(10)
fig, ax = plt.subplots(num=10, figsize=(9,6))

plt.loglog(1/freqs, ps)

fig.tight_layout()

In [ ]:
data = control[1][i:i+500]
data = data - np.mean(data)
c_ps = np.abs(np.fft.fft(data))**2

time_step = 1
c_freqs = np.fft.fftfreq(data.size, time_step)

c_ps = c_ps[1:len(c_ps)//2]
c_freqs = c_freqs[1:len(c_freqs)//2]

plt.close(10)
fig, ax = plt.subplots(num=10, figsize=(9,6))

plt.loglog(1/c_freqs, c_ps)

fig.tight_layout()

In [ ]:
plt.close(2)
fig, ax = plt.subplots(num=2, figsize=(9,6))

plt.loglog(1/c_freqs, c_ps, label=r'$\sigma = 0$')
plt.loglog(1/freqs, ps, label=r'$\sigma = 1$')

imx = np.argmax(c_ps)
plt.scatter(1/c_freqs[imx], c_ps[imx])

imx = np.argmax(ps)
plt.scatter(1/freqs[imx], ps[imx])

plt.legend()
plt.grid()

plt.xlabel('Period [yr]')
plt.ylabel('Power [$\mathrm{Sv}^2\mathrm{yr}$]')

fig.tight_layout()

In [ ]:
fig.savefig('../../papers/PhD_Thesis/manuscript/Figures/REA-VerOS/power-spectrum-v-sigma.pdf')